In [9]:
from smartcard import *
from smartcard.CardType import *
from smartcard.CardRequest import *
from smartcard.CardConnectionObserver import *
from smartcard.util import *
from smartcard.CardConnection import CardConnection
import binascii

In [2]:
cardtype = AnyCardType()
cardrequest = CardRequest( timeout=10, cardType=cardtype )
cardservice = cardrequest.waitforcard()

In [3]:
cardservice.connection.connect( )
print (toHexString( cardservice.connection.getATR() ))

3B 80 80 01 01


In [4]:
print(cardservice.connection.getReader())

ACS ACR1252 1S CL Reader [ACR1252 Dual Reader PICC] 00 00


In [5]:
SELECT = [0xA0, 0xA4, 0x00, 0x00, 0x02]
DF_TELECOM = [0x7F, 0x10]
BEEPOFF = [0xFF, 0x00,0x52,0x00,0x00]

In [6]:
SELECTFIDO = bytearray.fromhex('00a4040008a0000006472f000100')

In [7]:
list(SELECTFIDO)

[0, 164, 4, 0, 8, 160, 0, 0, 6, 71, 47, 0, 1, 0]

In [8]:
data, sw1, sw2 = cardservice.connection.transmit( list(SELECTFIDO))

In [9]:
[chr(x) for x in data] , hex(sw1), hex(sw2)

(['U', '2', 'F', '_', 'V', '2'], '0x90', '0x0')

In [10]:
data, sw1, sw2 = cardservice.connection.transmit( BEEPOFF )

In [11]:
U2F_register = bytearray.fromhex('''00 01 03 00 40 D1 7E 2D
D5 4D 12 B5 B8 24 DD CD
57 EC 01 3A D7 31 26 78
AF 0D 86 6C C0 35 3B AE
FF C5 67 92 D6 74 A6 EA
92 13 C9 9C 2F 74 B2 24
92 B3 20 CF 40 26 2A 94
C1 A9 50 A0 39 7F 29 25
0B 60 84 1E F0 00''')


In [14]:
data, sw1, sw2 = cardservice.connection.transmit( [0xFF, 0x00, 0x00,0x00, len(list(U2F_register))] + list(U2F_register) )

In [15]:
hex(sw1), hex(sw2)

('0x63', '0x0')

In [51]:
GETRESPONSE = list (bytearray.fromhex('80 C0 00 00 00'))

In [123]:
GETFIRMWAREVERSION = list (bytearray.fromhex('FF 00 48 00 00'))
data, sw1, sw2 = cardservice.connection.transmit( list(GETFIRMWAREVERSION) )
print (data, sw1, sw2)

[65, 67, 82, 49, 50, 50, 85, 50] 49 52


In [1]:
def turnoffbeep():
    BEEPOFF = [0xFF, 0x00,0x52,0x00,0x00]
    data, sw1, sw2 = cardservice.connection.transmit( BEEPOFF )

In [7]:
turnoffbeep()

In [106]:
TURNOFFTIMEOUT = [0xFF, 0x00,0x41,0xFF,0x00]
data, sw1, sw2 = cardservice.connection.transmit( list(TURNOFFTIMEOUT) )

In [1]:
import cbor2

In [3]:
cbordecodetest = bytearray.fromhex("""a5015820687134968222ec17202e42505f8ed2b16ae22f16bb05b88c25db9e602645f14102a26269646b6578616d706c652e636f6d646e616d656441636d6503a462696458203082019330820138a0030201023082019330820138a0030201023082019330826469636f6e782b68747470733a2f2f706963732e6578616d706c652e636f6d2f30302f702f61426a6a6a707150622e706e67646e616d65766a6f686e70736d697468406578616d706c652e636f6d6b646973706c61794e616d656d4a6f686e20502e20536d6974680482a263616c672664747970656a7075626C69632D6B6579a263616c6739010064747970656a7075626C69632D6B657907a162726bf5""")

In [6]:
obj = cbor2.loads(cbordecodetest)

In [12]:
obj

{1: b'hq4\x96\x82"\xec\x17 .BP_\x8e\xd2\xb1j\xe2/\x16\xbb\x05\xb8\x8c%\xdb\x9e`&E\xf1A',
 2: {'id': 'example.com', 'name': 'Acme'},
 3: {'id': b'0\x82\x01\x930\x82\x018\xa0\x03\x02\x01\x020\x82\x01\x930\x82\x018\xa0\x03\x02\x01\x020\x82\x01\x930\x82',
  'icon': 'https://pics.example.com/00/p/aBjjjpqPb.png',
  'name': 'johnpsmith@example.com',
  'displayName': 'John P. Smith'},
 4: [{'alg': -7, 'type': 'public-key'}, {'alg': -257, 'type': 'public-key'}],
 7: {'rk': True}}

In [13]:
obj[1].hex()

'687134968222ec17202e42505f8ed2b16ae22f16bb05b88c25db9e602645f141'

'687134968222ec17202e42505f8ed2b16ae22f16bb05b88c25db9e602645f141'